# Build an Agentic RAG Service

Setup an agent service that can interact with a tool service (containing RAG tools over annual reports).

In this notebook, we:
- Setup our indexes and query engine tools
- Define our multi-agent framework
  - A message queue.
  - An agentic orchestrator.
  - A tools service containing our query engine tools. This will act as a remote executor for tools
  - Define meta-tools for our agents. These will make calls to the tools service instead of executing directly
  - Our agent services. These wrap existing llama-index agents
  - Put all this into a local launcher, to simulate one task passing through the system at a time.

https://docs.llamaindex.ai/en/stable/examples/agent/openai_agent_tool_call_parser/

In [ ]:
%pip install llama-index-agent-openai
%pip install llama-index-llms-openai

In [ ]:
!pip install llama-index

In [ ]:
from google.colab import userdata


In [3]:
import nest_asyncio

nest_asyncio.apply()

import os

#os.environ["OPENAI_API_KEY"] = "sk-proj-..."
os.environ["OPENAI_API_KEY"] =userdata.get('open_ai_key')

## Load Data

In [ ]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

In [10]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [5]:
try:
    storage_context = StorageContext.from_defaults(persist_dir="./storage/lyft")
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(persist_dir="./storage/uber")
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

In [11]:
if not index_loaded:
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["./data/10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["./data/10k/uber_2021.pdf"]
    ).load_data()

    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs)
    uber_index = VectorStoreIndex.from_documents(uber_docs)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")

In [12]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

In [13]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

In [ ]:
!pip install llama_agents

## Setup Agents

Now that we've defined the query tools, we can wrap these under a `ToolService`.

In [16]:
from llama_agents import (
    AgentService,
    ToolService,
    LocalLauncher,
    MetaServiceTool,
    ControlPlaneServer,
    SimpleMessageQueue,
    AgentOrchestrator,
)

from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.llms.openai import OpenAI


# create our multi-agent framework components
message_queue = SimpleMessageQueue()
control_plane = ControlPlaneServer(
    message_queue=message_queue,
    orchestrator=AgentOrchestrator(llm=OpenAI(model="gpt-4o")),
)

# define Tool Service
tool_service = ToolService(
    message_queue=message_queue,
    tools=query_engine_tools,
    running=True,
    step_interval=0.5,
)

# define meta-tools here
meta_tools = [
    await MetaServiceTool.from_tool_service(
        t.metadata.name,
        message_queue=message_queue,
        tool_service=tool_service,
    )
    for t in query_engine_tools
]


# define Agent and agent service
worker1 = FunctionCallingAgentWorker.from_tools(
    meta_tools,
    llm=OpenAI(),
)
agent1 = worker1.as_agent()
agent_server_1 = AgentService(
    agent=agent1,
    message_queue=message_queue,
    description="Used to answer questions over Uber and Lyft 10K documents",
    service_name="uber_lyft_10k_analyst_agent",
)

## Launch agent

With our services, orchestrator, control plane, and message queue defined, we can test our llama-agents network by passing in single messages, and observing the results.

This is an excellent way to test, iterate, and debug your llama-agents system.

In [17]:
import logging

# change logging level to enable or disable more verbose logging
logging.getLogger("llama_agents").setLevel(logging.INFO)

In [18]:
## Define Launcher
launcher = LocalLauncher(
    [agent_server_1, tool_service],
    control_plane,
    message_queue,
)

In [ ]:
# query_str = "What was Lyft's revenue growth in 2021?"
# gets stuck in a loop, should mostly be called once
query_str = "What are the risk factors for Uber?"
result = launcher.launch_single(query_str)

In [20]:
print(result)

The risk factors for Uber include potential harm to reputation, business, and financial condition due to criminal, violent, inappropriate, or dangerous activities by platform users, inadequate background checks on Drivers and Couriers, incidents of criminal activity or misconduct by Drivers or consumers, liabilities from safety incidents, negative press coverage, data security breaches, challenges in introducing new products and features, regulatory complexities in the transportation industry, and legal issues related to driver classification and compliance with regulations such as AB5 in California.


In [21]:
query_str = "What was Lyft's revenue growth in 2021?"
result = launcher.launch_single(query_str)

INFO:llama_agents.message_queues.simple - Consumer AgentService-522ebebd-1f1e-47bf-8c85-8e6f51e92375: uber_lyft_10k_analyst_agent has been registered.
INFO:llama_agents.message_queues.simple - Consumer ToolService-356ebf9d-aed5-4bd8-878a-7c3709f5756c: default_tool_service has been registered.
INFO:llama_agents.message_queues.simple - Consumer fb2b16cc-f85c-40ea-afc3-8ca93ba1d5e9: human has been registered.
INFO:llama_agents.message_queues.simple - Consumer ControlPlaneServer-9e24641a-5bfa-4ae6-81f6-3daf790c7b5e: control_plane has been registered.
INFO:llama_agents.services.agent - uber_lyft_10k_analyst_agent launch_local
INFO:llama_agents.message_queues.base - Publishing message to 'control_plane' with action 'new_task'
INFO:llama_agents.message_queues.simple - Launching message queue locally
INFO:llama_agents.message_queues.base - Publishing message to 'uber_lyft_10k_analyst_agent' with action 'new_task'
INFO:llama_agents.message_queues.simple - Successfully published message 'control

In [22]:
print(result)

Lyft's revenue increased by 36% in 2021 compared to the prior year.
